In [3]:
import os
import glob
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
from sublimpy import utils, extrautils
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
import act
import swifter

Open tidy dataset

In [4]:
sos_tidy_fn = f"../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags360_pf10.parquet"
tidy_df = pd.read_parquet(sos_tidy_fn)
# Convert data timezone to local and clean up data on the ends
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')
# tidy_df = tidy_df[tidy_df.time > '20230301']
tidy_df = tidy_df[tidy_df.time > '20221130']
tidy_df = tidy_df[tidy_df.time < '20230509']
tidy_df.time.min(), tidy_df.time.max()

(Timestamp('2022-11-30 00:30:00'), Timestamp('2023-05-08 23:30:00'))

classify synoptic winds

In [5]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlprofwind4newsM1.c1'
DATA_STREAM_FILEEXT = '.nc'
startdate = "2022-11-30"
enddate = "2023-06-20"
output_dir = os.path.join("/Users/elischwat/Development/data/sublimationofsnow/", SAIL_DATA_STREAM)


In [6]:
# act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, startdate, enddate, output=output_dir)

In [7]:
dl_w_prof_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/*.nc")

In [8]:
prof_ds = act.io.read_arm_netcdf(dl_w_prof_files)
prof_ds.time.min(), prof_ds.time.max()

(<xarray.DataArray 'time' ()> Size: 8B
 array('2022-11-30T00:00:39.646250000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()> Size: 8B
 array('2023-06-15T23:59:20.789108000', dtype='datetime64[ns]'))

In [9]:
prof_df = prof_ds.to_dataframe()
prof_df = utils.modify_df_timezone(prof_df.reset_index(), 'UTC', 'US/Mountain').set_index('time')

In [10]:
hourly_mean_prof_df = prof_df.query("height > 950").query("height < 1100")[['wind_speed', 'wind_direction']].reset_index().set_index('time').groupby(pd.Grouper(freq='60min')).mean()

In [119]:
cross_valley_wind_times = pd.read_csv('cross_valley_wind_times.csv')
along_valley_wind_times = pd.read_csv('along_valley_wind_times.csv')
fast_synoptic_wind_times = pd.read_csv('winds_ge10_times.csv')
slow_synoptic_wind_times = pd.read_csv('winds_le10_times.csv')

In [120]:
cross_valley_wind_times = pd.DataFrame(
    cross_valley_wind_times['0'].rename('time')
).assign(category = 'cross-valley').set_index('time')
along_valley_wind_times = pd.DataFrame(
    along_valley_wind_times['0'].rename('time')
).assign(category = 'along-valley').set_index('time')
slow_synoptic_wind_times = slow_synoptic_wind_times.set_index('time').assign(synoptic_wind_speed = 'slow')[['synoptic_wind_speed']]
fast_synoptic_wind_times = fast_synoptic_wind_times.set_index('time').assign(synoptic_wind_speed = 'fast')[['synoptic_wind_speed']]

In [121]:
along_valley_wind_times.index = pd.to_datetime(along_valley_wind_times.index)
cross_valley_wind_times.index = pd.to_datetime(cross_valley_wind_times.index)
slow_synoptic_wind_times.index = pd.to_datetime(slow_synoptic_wind_times.index)
fast_synoptic_wind_times.index = pd.to_datetime(fast_synoptic_wind_times.index)

# Scaling, following Stull

In [122]:
scale_df = pd.DataFrame()
for H in [2,3,5,10,15,20]:
    src = tidy_df[tidy_df.variable.isin([
        f"w_w__{H}m_c",
        f"u*_{H}m_c",
    ])].pivot_table(index='time', columns='variable', values='value')
    src[f'scale'] = (
        src[f'w_w__{H}m_c'] / src[f'u*_{H}m_c']**2
    )
    src[f'height'] = H
    scale_df = pd.concat([scale_df, src[['height', 'scale']]])

In [123]:
scale_df = pd.concat([
    scale_df.join(along_valley_wind_times, how='inner'),
    scale_df.join(cross_valley_wind_times, how='inner')
])
scale_df = pd.concat([
    scale_df.join(fast_synoptic_wind_times, how='inner'),
    scale_df.join(slow_synoptic_wind_times, how='inner')
])

In [124]:
alt.layer(
        alt.Chart().mark_circle(size=150, color='#1f77b4').encode(
            alt.Color('height:O'),
            alt.X('category').axis(labelAngle=30).title(None),
            alt.XOffset('height:O'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.Color('height:O'),
            alt.X('category'),
            alt.XOffset('height:O'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df
).transform_filter(alt.datum.height == 5).facet(column=alt.Column('synoptic_wind_speed:N', title = 'Synoptic Wind Speed'))

alt.FacetChart(...)

In [125]:
alt.layer(
        alt.Chart().mark_circle(size=150, color='#1f77b4').encode(
            alt.Color('height:O'),
            alt.X('category').axis(labelAngle=30).title(None),
            alt.XOffset('height:O'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.Color('height:O'),
            alt.X('category'),
            alt.XOffset('height:O'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df
).properties().facet(column=alt.Column('synoptic_wind_speed:N', title = 'Synoptic Wind Speed'))

alt.FacetChart(...)

In [126]:
alt.layer(
        alt.Chart().mark_point(size=150, color='#1f77b4').encode(
            alt.X('category:N'),
            alt.Shape('category:N'),
            alt.Color('category:N'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.X('category:N').axis(values=[]).title(''),
            alt.Color('category:N'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
            detail = 'category'
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scale_df.query("synoptic_wind_speed == 'fast'").query("height > 2")
).properties(height=200).facet(column=alt.Column('height:O', title = 'Height AGL (m)'))


alt.FacetChart(...)

In [127]:
alt.layer(
        alt.Chart().mark_point(size=150, color='#1f77b4').encode(
            alt.X('date(time):N'),
            alt.Shape('date(time):N'),
            alt.Color('date(time):N'),
            alt.Y('median(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.X('date(time):N').axis(values=[]).title(''),
            alt.Color('date(time):N'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
            detail = 'date(time)'
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = pd.concat([ scale_df.loc['20230409'],scale_df.loc['20230418']]).reset_index().query("height > 2")
).properties(height=200).facet(column=alt.Column('height:O', title = 'Height AGL (m)'))


alt.FacetChart(...)

In [128]:
scaling_src_case_studies = pd.concat([ scale_df.loc['20230409'],scale_df.loc['20230418']]).reset_index().query("height > 2")
scaling_src_case_studies['category'] = scaling_src_case_studies.time.dt.date.astype('str')
scaling_src_case_studies_and_all_data = pd.concat([
    scale_df.query("synoptic_wind_speed == 'fast'").query("height > 2"),
    scaling_src_case_studies
])
scaling_src_case_studies_and_all_data.category = scaling_src_case_studies_and_all_data.category.replace({
    '2023-04-18': 'cross-valley (just 2023-04-18)',
    '2023-04-09': 'along-valley (just 2023-04-09)',
})

In [133]:
alt.layer(
        alt.Chart().mark_point(size=150, color='#1f77b4').encode(
            alt.X('category:N'),
            alt.Shape('category:N'),
            alt.Color('category:N').scale(
                domain = [
                    'along-valley', 'along-valley (just 2023-04-09)',
                    'cross-valley', 'cross-valley (just 2023-04-18)'
                ],
                range = ['#1f77b4', '#1f77b4', '#ff7f0e', '#ff7f0e']
            ),
            alt.Y('mean(scale):Q').title("w'w' / u*"),
        ),
        alt.Chart().mark_line(color='#1f77b4', strokeWidth=2).encode(
            alt.X('category:N').axis().title(''),
            alt.Color('category:N'),
            alt.Y('q1(scale)'),
            alt.Y2('q3(scale):Q'),
            detail = 'category'
        ),
        alt.Chart().transform_calculate(
            y = '2.5'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        alt.Chart().transform_calculate(
            y = '1.7'
        ).mark_rule(color='black').encode(alt.Y('y:Q')),
        data = scaling_src_case_studies_and_all_data.query("height < 10")
).properties(height=200).facet(column=alt.Column('height:O', title = 'Height AGL (m)'))


alt.FacetChart(...)

alt.FacetChart(...)

In [71]:
src = tidy_df.query("measurement == 'w_h2o_'").query("tower == 'c'").query("height > 2")
src = src[src.time.dt.date.astype('str') == '2023-04-18']
src = src[~src.variable.str.contains('gapfill')]
alt.Chart(src).mark_line().encode(
    alt.X('mean(value):Q'),
    alt.Y('height'),
    alt.Facet('hours(time):O', columns=6),
    order = 'height'
).properties(width=100, height=100)

alt.Chart(...)

In [72]:
src = tidy_df.query("measurement == 'w_tc_'").query("tower == 'c'").query("height > 2")
src = src[src.time.dt.date.astype('str') == '2023-04-18']
src = src[~src.variable.str.contains('gapfill')]
alt.Chart(src).mark_line().encode(
    alt.X('mean(value):Q'),
    alt.Y('height'),
    alt.Facet('hours(time):O', columns=6),
    order = 'height'
).properties(width=100, height=100)

alt.Chart(...)

In [74]:
src = tidy_df.query("measurement == 'shear velocity'").query("tower == 'c'").query("height > 2")
src = src[src.time.dt.date.astype('str') == '2023-04-18']
src = src[~src.variable.str.contains('gapfill')]
alt.Chart(src).mark_line().encode(
    alt.X('mean(value):Q'),
    alt.Y('height'),
    alt.Facet('hours(time):O', columns=6),
    order = 'height'
).properties(width=100, height=100)

alt.Chart(...)